# Iterated Prisoner's Dilemma On A Network

## Imports and Config

In [ ]:
import os
import math
import random
import logging
import numpy as np
import pandas as pd
import networkx as nx
from functools import partial
from dataclasses import dataclass
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
from matplotlib.ticker import PercentFormatter
from matplotlib.animation import FuncAnimation

In [ ]:
%matplotlib notebook
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams["animation.embed_limit"] = 500

In [ ]:
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(name)s - %(message)s",
)

## Matrices

In [ ]:
payoff_matrices = {  # some of these don't look right
    "Default": {
        ("C", "C"): (3, 3),
        ("C", "D"): (0, 5),
        ("D", "C"): (5, 0),
        ("D", "D"): (0, 0),
    },
    "Canonical": {
        ("C", "C"): (-1, -1),
        ("C", "D"): (-3, 0),
        ("D", "C"): (0, -3),
        ("D", "D"): (-2, -2),
    },
    "Friend or Foe": {
        ("C", "C"): (1, 1),
        ("C", "D"): (0, 2),
        ("D", "C"): (2, 0),
        ("D", "D"): (0, 0),
    },
    "Snowdrift": {
        ("C", "C"): (500, 500),
        ("C", "D"): (200, 800),
        ("D", "C"): (800, 200),
        ("D", "D"): (0, 0),
    },
    "Prisoners": {
        ("C", "C"): (500, 500),
        ("C", "D"): (-200, 1200),
        ("D", "C"): (1200, -200),
        ("D", "D"): (0, 0),
    },
}

## Classes

In [ ]:
from Prison import ActionStrategy, ImitationStrategy, FermiStrategy, ReinforcementLearningStrategy, TitForTatStrategy
from Prison import Agent, Network, NetworkSimulation
from Prison import experiment
from Prison import jaccard_sets, jaccard_indices, cluster_sizes

## Helpers

In [ ]:
def save_plot(fig, filename, output_dir="Results/Result_RQ2"):
    os.makedirs(output_dir, exist_ok=True)
    fig.savefig(os.path.join(output_dir, filename), dpi=200, bbox_inches="tight")
    plt.close(fig)

### Params


In [ ]:
NETWORKS = {
    "Grid": dict(kind="grid", n=400),
    "WS (p=0.1)": dict(kind="watts_strogatz", n=400, k=4, p=0.1),
    "ER": dict(kind="erdos_renyi", n=400, p=0.05),
}

In [ ]:
STRATEGIES = {
    "TFT": TitForTatStrategy,
    "Imitation": ImitationStrategy,
    "RL": ReinforcementLearningStrategy,
}

PAYOFFS = {
    "Canonical": payoff_matrices["Canonical"],
    "Default": payoff_matrices["Default"],
    "Snowdrift": payoff_matrices["Snowdrift"],
}

### General computing

In [ ]:
def collect_cluster_sizes_window(results, t_start, t_end):
    sizes = []
    for r in results:
        for step_sizes in r["sizes"][t_start:t_end]:
            sizes.extend(step_sizes)
    return sizes

In [ ]:
def collect_Smax_vs_c(results, Tburn):
    xs, ys = [], []
    for r in results:
        xs.extend(r["c"][Tburn:])
        ys.extend(r["Smax"][Tburn:])
    return np.asarray(xs), np.asarray(ys)


In [ ]:
def make_time_windows(L, Tburn):
    T_eff = L - Tburn
    if T_eff <= 0:
        return {}

    return {
        "Early": (
            Tburn,
            Tburn + max(1, int(0.1 * T_eff)),
        ),
        "Mid": (
            Tburn + int(0.4 * T_eff),
            Tburn + int(0.6 * T_eff),
        ),
        "Late": (
            Tburn + max(0, int(0.9 * T_eff)),
            L,
        ),
    }

### Plotting

In [ ]:
def plot_Smax_across_networks(network_results, strategy, payoff, filename=None):
    fig, ax = plt.subplots(figsize=(6, 4))
    for net_name, data in network_results.items():
        ts = data["agg"]["Smax"]
        t = np.arange(len(ts["mean_ts"]))
        ax.plot(t, ts["mean_ts"], lw=2, label=net_name)
        ax.fill_between(t, ts["mean_ts"] - ts["std_ts"], ts["mean_ts"] + ts["std_ts"], alpha=0.2)

    ax.set_xlabel("Time step")
    ax.set_ylabel("Largest cooperative cluster fraction $S_{\\max}(t)$")
    ax.set_title(f"$S_{{\\max}}(t)$ across networks\n({strategy}, {payoff})")
    ax.set_ylim(0, 1)
    ax.grid(alpha=0.3)
    ax.legend(frameon=False)
    plt.tight_layout()
    if filename:
        save_plot(fig, filename)
    else:
        plt.show()

In [ ]:
def plot_JV_vs_c_time_colored(
    network_results,
    strategy,
    payoff,
    Tburn,
    filename=None,
    cmap="plasma",
):
    fig, ax = plt.subplots(figsize=(5, 5))

    for net_name, data in network_results.items():
        xs, ys, ts = [], [], []

        for r in data["raw"]:
            L = len(r["c"])
            tvals = np.arange(L)[Tburn:]

            xs.extend(r["c"][Tburn:])
            ys.extend(r["JV"][Tburn:])
            ts.extend(tvals)

        ts = np.asarray(ts)
        ts_norm = (ts - ts.min()) / (ts.max() - ts.min() + 1e-12)

        sc = ax.scatter(
            xs,
            ys,
            c=ts_norm,
            cmap=cmap,
            s=10,
            alpha=0.25,
        )

    ax.set_xlabel("Cooperation fraction $|V_C|/N$")
    ax.set_ylabel("Cooperator set stability $J_V$")
    ax.set_title(
        "Evolution of cooperation stability\n"
        f"({strategy}, {payoff})"
    )
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.grid(alpha=0.3)

    cbar = plt.colorbar(sc, ax=ax)
    cbar.set_label("Normalized time")

    plt.tight_layout()
    if filename:
        save_plot(fig, filename)
    else:
        plt.show()

In [ ]:
def plot_JV_time_series_across_networks(network_results, strategy, payoff, filename=None):
    fig, ax = plt.subplots(figsize=(6, 4))

    for net_name, data in network_results.items():
        ts = data["agg"]["JV"]
        t = np.arange(len(ts["mean_ts"]))

        ax.plot(t, ts["mean_ts"], lw=2, label=net_name)
        ax.fill_between(
            t,
            ts["mean_ts"] - ts["std_ts"],
            ts["mean_ts"] + ts["std_ts"],
            alpha=0.25,
        )

    ax.set_xlabel("Time step")
    ax.set_ylabel("Cooperator set stability $J_V(t)$")
    ax.set_title(
        "Stability of the cooperator set over time\n"
        f"({strategy}, {payoff})"
    )
    ax.set_ylim(0, 1)
    ax.grid(alpha=0.3)
    ax.legend(frameon=False)

    plt.tight_layout()
    if filename:
        save_plot(fig, filename)
    else:
        plt.show()

In [ ]:
def plot_JL_time_series_across_networks(network_results, strategy, payoff, filename=None):
    fig, ax = plt.subplots(figsize=(6, 4))

    for net_name, data in network_results.items():
        ts = data["agg"]["JL"]
        t = np.arange(len(ts["mean_ts"]))

        ax.plot(t, ts["mean_ts"], lw=2, label=net_name)
        ax.fill_between(
            t,
            ts["mean_ts"] - ts["std_ts"],
            ts["mean_ts"] + ts["std_ts"],
            alpha=0.25,
        )

    ax.set_xlabel("Time step")
    ax.set_ylabel("Largest cluster stability $J_L(t)$")
    ax.set_title(
        "Stability of the largest cooperative cluster\n"
        f"({strategy}, {payoff})"
    )
    ax.set_ylim(0, 1)
    ax.grid(alpha=0.3)
    ax.legend(frameon=False)

    plt.tight_layout()
    if filename:
        save_plot(fig, filename)
    else:
        plt.show()


In [ ]:
def plot_time_resolved_cluster_distributions(
    network_results,
    strategy,
    matrix,
    Tburn,
    filename=None,
):
    L = min(len(r["sizes"]) for net in network_results.values() for r in net["raw"])
    windows = make_time_windows(L, Tburn)


    markers = {
        "Early": "o",
        "Mid":   "s",
        "Late":  "^",
    }

    colors = {
        "Grid": "tab:blue",
        "WS (p=0.1)": "tab:orange",
        "ER": "tab:green",
    }

    fig, ax = plt.subplots(figsize=(7, 5))

    for net_name, net_data in network_results.items():
        results = net_data["raw"]
        color = colors.get(net_name, None)

        for stage, (t0, t1) in windows.items():
            sizes = collect_cluster_sizes_window(results, t0, t1)

            
            sizes = np.array(sizes)
            sizes = sizes[sizes >= 1]

            if len(sizes) == 0:
                continue

            bins = np.logspace(
                np.log10(1),
                np.log10(max(sizes)),
                20,
            )

            counts, edges = np.histogram(
                sizes,
                bins=bins,
                density=True,
            )
            centers = 0.5 * (edges[:-1] + edges[1:])

            mask = counts > 0
            centers = centers[mask]
            counts = counts[mask]

            ax.plot(
                centers,
                counts,
                linestyle="",
                marker=markers[stage],
                color=color,
                alpha=0.7,
                label=f"{net_name} – {stage}",
            )

    ax.set_xscale("log")
    ax.set_yscale("log")

    ax.set_xlabel("Cooperative cluster size $s$")
    ax.set_ylabel("Probability density $P(s)$")

    ax.set_title(
        "Time-resolved cooperative cluster size distributions (log–log)\n"
        f"({strategy}, {matrix})"
    )

    ax.grid(True, which="both", alpha=0.3)

   
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), fontsize=8)

    plt.tight_layout()

    if filename:
        save_plot(fig, filename)

In [ ]:
def plot_Smax_vs_c_time_colored(
    network_results,
    strategy,
    payoff,
    Tburn,
    filename=None,
    cmap="viridis",
):
    fig, ax = plt.subplots(figsize=(5, 5))

    all_t = []

    for net_name, data in network_results.items():
        xs, ys, ts = [], [], []

        for r in data["raw"]:
            L = len(r["c"])
            tvals = np.arange(L)[Tburn:]

            xs.extend(r["c"][Tburn:])
            ys.extend(r["Smax"][Tburn:])
            ts.extend(tvals)

        ts = np.asarray(ts)
        ts_norm = (ts - ts.min()) / (ts.max() - ts.min() + 1e-12)

        sc = ax.scatter(
            xs,
            ys,
            c=ts_norm,
            cmap=cmap,
            s=10,
            alpha=0.25,
            label=net_name,
        )

        all_t.append(ts_norm)

    ax.plot([0, 1], [0, 1], "k--", alpha=0.4)
    ax.set_xlabel("Overall cooperation fraction $|V_C|/N$")
    ax.set_ylabel("Largest cluster fraction $S_{\\max}$")
    ax.set_title(
        "Evolution of cluster dominance vs cooperation\n"
        f"({strategy}, {payoff})"
    )
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.grid(alpha=0.3)

    cbar = plt.colorbar(sc, ax=ax)
    cbar.set_label("Normalized time")

    plt.tight_layout()
    if filename:
        save_plot(fig, filename)
    else:
        plt.show()

## Run

In [ ]:
def run_once(seed, **kwargs):
    m = NetworkSimulation(seed=seed, **kwargs)
    m.run()
    return {
        "Smax": np.asarray(m.Smax, float),
        "c":    np.asarray(m.c, float),
        "JL":   np.asarray(m.JL, float),
        "JV":   np.asarray(m.JV, float),
        "sizes": m.cluster_sizes_ts,
    }

In [ ]:
def aggregate_runs(results, Tburn=0):
    keys = ["Smax", "c", "JL", "JV", "Neff", "B"]
    L = min(len(r[k]) for r in results for k in keys)
    out = {}
    for k in keys:
        X = np.stack([r[k][:L] for r in results], axis=0)
        out[k] = {
            "mean_ts": X.mean(axis=0),
            "std_ts":  X.std(axis=0, ddof=1),
            "mean_scalar": X[:, Tburn:].mean(),
        }
    out["_L"] = L
    return out

In [ ]:
def run_simulation(seeds=range(2), rounds=100, Tburn=50, save=True):
    for strategy_name, strategy_class in STRATEGIES.items():
        for payoff_name, payoff_matrix in PAYOFFS.items():
            # print(f"\nRunning: {strategy_name} | {payoff_name}")

            network_results = {}
            for net_name, net_cfg in NETWORKS.items():
                print(f"  Network: {net_name}")
                results = [
                    run_once(
                        seed=s,
                        rounds=rounds,
                        strategy=strategy_class,
                        payoff_matrix=payoff_matrix,
                        **net_cfg,
                    )
                    for s in seeds
                ]
                network_results[net_name] = {
                    "raw": results,
                    "agg": aggregate_runs(results, Tburn=Tburn),
                }

            # filenames (optional)
            prefix = f"{strategy_name}_{payoff_name}".replace(" ", "").replace("(", "").replace(")", "")
            fn1 = f"{prefix}_Smax_across_networks.png" if save else None
            fn2 = f"{prefix}_Smax_vs_c_across_networks.png" if save else None
            fn3 = f"{prefix}_cluster_dist_time.png" if save else None
            fn4 = f"{prefix}_JV_vs_c.png" if save else None
            fn5 = f"{prefix}_JV_time.png" if save else None
            fn6 = f"{prefix}_JL_time.png" if save else None



            plot_Smax_across_networks(network_results, strategy_name, payoff_name, filename=fn1)
            plot_Smax_vs_c_time_colored(network_results, strategy_name, payoff_name, Tburn=Tburn, filename=fn2)
            plot_time_resolved_cluster_distributions( network_results, strategy_name, payoff_name, Tburn=Tburn, filename=fn3)
            plot_JV_vs_c_time_colored(network_results,strategy_name,payoff_name, Tburn=Tburn,filename=fn4)
            plot_JV_time_series_across_networks(network_results,strategy_name,payoff_name,filename=fn5)
            plot_JL_time_series_across_networks(network_results,strategy_name,payoff_name,filename=fn6,)

In [ ]:
run_simulation()